In [1]:
import json
import re
import requests
import unicodedata
import time
import os
import subprocess
import codecs

# Diff


In [13]:
# trim BOM
for name in os.listdir("../data/www/data/"):
    if name.endswith(".json"):
        with open("../data/www/data/" + name, "rb") as f:
            data = f.read()
        with open("nobom/data/" + name, "wb") as f:
            f.write(data.replace(codecs.BOM_UTF8, b"", 1))
for name in os.listdir("../data/www/data_cn/"):
    if name.endswith(".json"):
        with open("../data/www/data_cn/" + name, "rb") as f:
            data = f.read()
        with open("nobom/data_cn/" + name, "wb") as f:
            f.write(data.replace(codecs.BOM_UTF8, b"", 1))

In [14]:
for name in os.listdir("nobom/data_cn"):
    if not name.endswith(".json"):
        continue
    cmd = f'jd-amd64-windows.exe -f patch -o diff/{name} {"nobom/data/"+name} {"nobom/data_cn/"+name}'
    p = subprocess.Popen(
        cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )
    stdout, stderr = p.communicate()
    print(name, stdout, stderr)

Actors.json b'' b''
Animations.json b'' b''
Armors.json b'' b''
Classes.json b'' b''
CommonEvents.json b'' b''
Enemies.json b'' b''
Items.json b'' b''
Map001.json b'' b''
Map002.json b'' b''
Map003.json b'' b''
Map004.json b'' b''
Map005.json b'' b''
Map006.json b'' b''
Map007.json b'' b''
Map008.json b'' b''
Map009.json b'' b''
Map010.json b'' b''
Map011.json b'' b''
Map012.json b'' b''
Map013.json b'' b''
Map014.json b'' b''
Map015.json b'' b''
Map016.json b'' b''
Map017.json b'' b''
Map018.json b'' b''
Map019.json b'' b''
Map020.json b'' b''
Map021.json b'' b''
Map022.json b'' b''
Map023.json b'' b''
Map024.json b'' b''
Map025.json b'' b''
Map026.json b'' b''
Map027.json b'' b''
Map028.json b'' b''
Map029.json b'' b''
Map030.json b'' b''
Map031.json b'' b''
Map032.json b'' b''
Map033.json b'' b''
Map034.json b'' b''
Map035.json b'' b''
Map036.json b'' b''
Map037.json b'' b''
Map038.json b'' b''
Map039.json b'' b''
Map040.json b'' b''
Map041.json b'' b''
Map042.json b'' b''
Map043.js

# Extract


In [19]:
for file in os.listdir("diff"):
    with open("diff/" + file, "r", encoding="utf-8") as f:
        patch = json.load(f)
        patch2 = []
        for i in range(0, len(patch), 3):
            assert patch[i]["op"] == "test"
            assert patch[i + 1]["op"] == "remove"
            assert patch[i + 2]["op"] == "add"
            path = patch[i]["path"]
            jp = patch[i + 1]["value"]
            cn = patch[i + 2]["value"]
            if type(cn) != str or type(jp) != str:
                continue
            if re.match(r"^(\\N<.+>)?$", cn) and re.match(r"^(\\N<.+>)?[^>]+$", jp):
                patch2.append({"path": path, "jp": jp, "cn": cn})
        if len(patch2) == 0:
            continue
        with open("dlc/" + file, "w", encoding="utf-8") as f2:
            json.dump(patch2, f2, ensure_ascii=False, indent=2)
        print(file, len(patch2))

CommonEvents.json 32
Map001.json 4470
Map002.json 27
Map005.json 0
Map006.json 4
Map007.json 15
Map008.json 3
Map011.json 0
Map017.json 63
Map018.json 11
Map019.json 18
Map020.json 2
Map021.json 13
Map022.json 1
Map023.json 33
Map024.json 13
Map025.json 1
Map026.json 0
Map027.json 1
Map028.json 0
Map029.json 3
Map030.json 1
Map032.json 18
Map033.json 25
Map034.json 11
Map036.json 16
Map037.json 3
Map038.json 1
Map039.json 0
Map040.json 4
Map041.json 0
Map042.json 2
Map043.json 0
Map044.json 4
Map045.json 0
Map046.json 0
Map048.json 0
Map049.json 3
Map050.json 1
Map051.json 5
Map057.json 7
Map060.json 3
Map061.json 0
Map062.json 4
Map063.json 5
Map064.json 10
Map065.json 0
Map066.json 4
Map067.json 1
Map068.json 3
Map069.json 1
Map070.json 14
Map071.json 0
Map073.json 3
Map074.json 2
Map076.json 8
Map077.json 10
Map078.json 0
Map079.json 2
Map081.json 8
Map082.json 4
Map083.json 7
Map084.json 13
Map085.json 0
Map087.json 2
Map088.json 0
Map092.json 4
Map094.json 0
Map095.json 0
Map096.j

# Join breaking sentences

In [114]:
def join_breaking_event_sentences(map_id):
    sentences = None
    with open(f"dlc/Map{map_id}.json", "r", encoding="utf-8") as f:
        sentences = json.load(f)
        pattern = re.compile(
            r"/events/(\d+)/pages/(\d+)/list/(\d+)/parameters/0/?(\d)?"
        )

    for sentence in sentences:
        events_id, pages_id, list_id, extra = pattern.match(sentence["path"]).groups(-1)
        sentence["events_id"] = int(events_id)
        sentence["list_id"] = int(list_id)
        sentence["pages_id"] = int(pages_id)
        sentence["extra"] = int(extra)
    sentences.sort(
        key=lambda x: [x["extra"], x["pages_id"], x["events_id"], x["list_id"]]
    )

    merged = []
    prev_events_id = 0
    prev_list_id = 0
    current = {}
    for sentence in sentences:
        events_id = sentence["events_id"]
        list_id = sentence["list_id"]
        if events_id != prev_events_id or prev_list_id + 1 != list_id:
            current = {
                "events_id": events_id,
                "list_id": [],
                "jp": [],
                "cn": [],
                "pages_id": sentence["pages_id"],
                "extra": sentence["extra"],
            }
            merged.append(current)

        current["jp"].append(sentence["jp"])
        current["cn"].append(sentence["cn"])
        current["list_id"].append(list_id)

        prev_events_id = events_id
        prev_list_id = list_id


    with open(f"dlc/Map{map_id}.join.json", "w", encoding="utf-8") as f:
        json.dump(merged, f, ensure_ascii=False, indent=2)

    print(f"Map{map_id}", len(merged))


for file in os.listdir("dlc"):
    if not file.startswith("Map"):
        continue
    if (m := re.match(r"Map(\d{3})\.json", file)) == None:
        continue
    join_breaking_event_sentences(m.group(1))

Map001 3261
Map002 25
Map006 4
Map007 14
Map008 3
Map017 60
Map018 11
Map019 17
Map020 2
Map021 12
Map022 1
Map023 33
Map024 13
Map025 1
Map027 1
Map029 3
Map030 1
Map032 18
Map033 25
Map034 7
Map036 14
Map037 3
Map038 1
Map040 2
Map042 2
Map044 4
Map049 3
Map050 1
Map051 5
Map057 7
Map060 3
Map062 4
Map063 5
Map064 9
Map066 4
Map067 1
Map068 3
Map069 1
Map070 14
Map073 3
Map074 1
Map076 8
Map077 10
Map079 2
Map081 8
Map082 4
Map083 7
Map084 13
Map087 2
Map092 4
Map098 1
Map103 3
Map111 2
Map113 5
Map114 1
Map121 3
Map127 1
Map137 2
Map143 3
Map144 3
Map145 2
Map146 4
Map147 2
Map148 8
Map149 7
Map150 10
Map152 2
Map154 2
Map158 4
Map160 11
Map164 1
Map166 1
Map168 4
Map173 1
Map174 7
Map175 1
Map178 3
Map183 11
Map184 5
Map186 2
Map187 4
Map188 6
Map190 2
Map191 4
Map192 9
Map193 1
Map194 23
Map195 8
Map198 3
Map199 1
Map200 2
Map201 8
Map202 2
Map203 1
Map207 19


# Translate


In [94]:
# cf_clearance = ''
def deeplx(s):
    url = "https://api.deeplx.org/translate"

    payload = {
        "text": s,
        "source_lang": "JP",
        "target_lang": "ZH",
    }
    headers = {
        "Content-Type": "application/json",
        "Cookie": f"cf_clearance={cf_clearance}",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0",
    }
    resp = requests.post(url, headers=headers, data=json.dumps(payload)).json()['data']
    return resp

In [69]:
# qwen_api_url = ''
def qwen(s):
    headers = {
        "Content-Type": "application/json",
    }
    response = requests.post(
        f"{qwen_api_url}/trans",
        data=json.dumps({"text": s}),
        headers=headers,
    )
    return response.text.replace('\\', '')

In [83]:
import jieba


def translate_wrap(jp):
    jp_join = re.match(r"^(?:\\N<.+>)?([^>]+$)", "".join(jp)).group(1)

    cn = deeplx(unicodedata.normalize("NFKC", jp_join.strip("「」")))
    cn = cn.strip('"”“')
    if jp_join.startswith("「"):
        cn = "「" + cn
    if jp_join.endswith("」"):
        cn += "」"

    cut = list(jieba.cut(cn))
    line_length = []
    for ratio in map(lambda s: len(s) / len("".join(jp)), jp):
        line_length.append(int(ratio * len(cn)))

    cn_wrap = [""] * len(jp)
    idx = 0
    for i in range(len(cn_wrap) - 1):
        while len(cn_wrap[i]) < line_length[i]:
            cn_wrap[i] += cut[idx]
            idx += 1
    cn_wrap[-1] = "".join(cut[idx:])
    return cn_wrap

In [89]:
jp = [
    "\\N<ルミ>経験値のかけらを濃縮してなんか",
      "アレしてコレして作るんだってさ",
]

print(translate_wrap(jp))

['他们把经验集中起来', '，做这做那。']


In [116]:
def translate(sentences, start_idx = 0):
  start_time = time.time()
  for i in range(start_idx, len(sentences)):
    t = sentences[i]

    cn = ''.join(t['cn'])
    prefix = re.match(r'^(\\N<.+>)?', cn).groups('')[0]

    # cn_wrap = translate_wrap(t['jp'])
    cn_wrap = t['jp']

    cn_wrap[0] = prefix + cn_wrap[0]
    t['cn'] = cn_wrap

    print(f'{int(time.time()-start_time):05} {i}/{len(sentences)} {t['jp']} -> {t['cn']}')


In [ ]:
with open("dlc/Map001.join.json", "r", encoding="utf-8") as f:
    sentences = json.load(f)
    try:
        translate(sentences)
    finally:
        with open('dlc_cn/Map001.join.json', 'w', encoding='utf-8') as f2:
            json.dump(sentences, f2, ensure_ascii=False, indent=2)

In [ ]:
for file in os.listdir("dlc"):
    if not file.endswith(".join.json"):
        continue
    if file == "Map001.join.json":
        continue
    print("*" * 20 + file + "*" * 20)
    with open("dlc/"+file, "r", encoding="utf-8") as f:
        sentences = json.load(f)
        try:
            translate(sentences)
        finally:
            with open('dlc_cn/'+file, 'w', encoding='utf-8') as f2:
                json.dump(sentences, f2, ensure_ascii=False, indent=2)


# Better Sentence Wrap

# Merge


In [129]:
for file in os.listdir("dlc_cn"):
    if (m := re.match(r"(.+)\.join\.json", file)) == None:
        continue
    origin = f"../data/www/data_cn/{m.group(1)}.json"
    with open(f'nobom/data_cn/{m.group(1)}.json', "r", encoding="utf-8") as f1, open(
        "dlc_cn/" + file, "r", encoding="utf-8"
    ) as f2:
        data = json.load(f1)
        trans = json.load(f2)
        for entry in trans:
            for i, list_id in enumerate(entry['list_id']):
                def set_(v):
                    if entry['extra'] != -1:
                        data["events"][entry["events_id"]]["pages"][entry["pages_id"]]['list'][list_id]['parameters'][0][entry['extra']] = v
                    else:
                        data["events"][entry["events_id"]]["pages"][entry["pages_id"]]['list'][list_id]['parameters'][0] = v
                set_(entry['cn'][i])
        with open(origin, 'w', encoding='utf-8-sig') as f3:
            json.dump(data, f3, ensure_ascii=False)